In [ ]:
'''
Mixture density neural network with 4 hidden dense layers 
and 1 mixture density layer to predict output distribution.
Preprocessing code written by Prof. Horvat, network code written by Meal Swipes for Late Days.
Input is a (batch_size x 26) tensor representing the input wave spectrum, 
output is a (batch size x 49) tensor representing the output floe size distribution.
'''
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.io import loadmat,savemat
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow import keras
import tensorflow_probability as tfp
import random
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping
from google.colab import drive 

In [ ]:
# Define file locations

usable_class = 0

include_last = True;

drive.mount('/content/drive', force_remount=True)
# locstr = '/Users/chorvat/Dropbox (Brown)/Research Projects/Active/FSTD-Ongoing-Work/Neural-Net-Waves/'
locstr = '/content/drive/Shared drives/DeepWIFF/'

training_string = locstr + 'Data/training_data_errtol'

if not include_last:
    training_string = training_string + '_nolast'

training_string = training_string + '.mat'


Mounted at /content/drive


In [ ]:
'''
Loads training data and performs preprocessing to remove degenerate inputs.
There is no need to run this cell once the pickle files have been saved once.
'''

training_data = loadmat(training_string)
misc_data = loadmat(locstr + 'Data/NN_params.mat')

input_vec = training_data['input_vec'].T
output_vec = training_data['output_vec'].T

Freq = misc_data['f']
dFreq = Freq * (np.sqrt(1.1) - np.sqrt(1/1.1)); 
Redge = misc_data['redge'].T
Rcent = misc_data['rcent'].T
Lambda = misc_data['lambda']


num_frac = np.sum(output_vec,axis=1)
# classifier = load_model('Classifiers/KERAS-Classifier-25x25.h5')
# thresh = 0.11;
# y_pred_class = classifier.predict(input_vec)
# usable = y_pred_class[:,1] > thresh

usable = num_frac > 0
print('Out of ' + str(len(num_frac)) + ', ' + str(sum(usable)) + ' can be used')

ice_thick = input_vec[:,-1]
wave_spec = input_vec[:,:25]

# wave_spec[wave_spec < 10**-8] = 0
wave_energy = np.sum(wave_spec*np.expand_dims(dFreq,axis=1).T,axis=1)
peak_loc = np.argmax(wave_spec,axis=1)
peak_freq = Freq[peak_loc]

NN_input = input_vec[usable]
NN_output = output_vec[usable]



Out of 1830900, 404503 can be used


In [ ]:
'''
Loads training and testing data from pickle file.
'''
import pickle

try:

    X_train = pickle.load( open(locstr +  "Data/" + "X_train.p", "rb" ) )
    X_test = pickle.load( open( locstr + "Data/" + "X_test.p", "rb" ) )
    y_train = pickle.load( open(locstr + "Data/" + "y_train.p","rb"))
    y_test = pickle.load( open(locstr + "Data/" + "y_test.p","rb"))
    
except:
        
        print('exception in loading segmented data')
        X_train, X_test, y_train, y_test = train_test_split(NN_input,NN_output,test_size=0.3)
        
        pickle.dump(X_train, open( locstr + "Data/" + "X_train.p", "wb" ) )
        pickle.dump(X_test, open( locstr + "Data/" + "X_test.p", "wb" ) )
        pickle.dump(y_train, open( locstr + "Data/" + "y_train.p", "wb" ) )
        pickle.dump(y_test, open( locstr + "Data/" + "y_test.p", "wb" ) )

In [ ]:
tf.print("X_train mean: ", tf.math.reduce_mean(X_train))
tf.print("X_train stdev: ", tf.math.reduce_std(X_train))
tf.print("y_train mean: ", tf.math.reduce_mean(y_train))
tf.print("y_train stdev: ", tf.math.reduce_std(y_train))

X_train mean:  1.1930075366304653
X_train stdev:  4.3379647876911962
y_train mean:  0.020408163265306121
y_train stdev:  0.066589406428670186


In [ ]:
# Normalize input vectors featurewise (or don't)
def featureNorm(data):
    mean = tf.expand_dims(tf.math.reduce_mean(data, axis = 1), axis = 1)
    std = tf.expand_dims(tf.math.reduce_std(data, axis = 1), axis = 1)
    return tf.math.divide(tf.math.subtract(data, mean), std)

#X_train = featureNorm(X_train)
#X_test = featureNorm(X_test)

In [ ]:
#Mixture density layer definition.
tf.keras.backend.set_floatx('float64')
class MixtureDensity(tf.keras.layers.Layer):
    # This layer generates a mixture distribution learned from a batch of 1d feature vectors
    def __init__(self, num_components, input_dim):
        """
		Intializer for a MixtureDensity layer. Creates layer and initializes the trainable variables as keras Dense layers,
        with respective activation functions.

		:param num_components:  integer tensor, denoting the number of learned Gaussian distributions mixed together
		:param input_dim:  integer tensor, the 1D input dimension of the values input into the MixtureDensity layer
		"""

        # data_mean and data_std are set the mean value for the mu and sigma networks respectively
        super(MixtureDensity, self).__init__()
        
        self.regularizer = {} #{'kernel_regularizer': keras.regularizers.l2(0.001), 'bias_regularizer': keras.regularizers.l2(0.001)}
        self.num_components = num_components
        
        # Dense layer to generate mixing parameters (softmax ensures that alpha values sum to 1)
        self.alpha = keras.layers.Dense(num_components, activation = 'softmax', **self.regularizer)
        
        #  Dense layer to generate mean parameters
        self.mu = keras.layers.Dense(num_components, **self.regularizer)
        
        # Dense layer to generate standard deviation parameters (reluing to debug invalid parameter outputs)
        self.sigma = keras.layers.Dense(num_components, activation= 'softplus', **self.regularizer)

    def call(self, inputs):
         """
		Call function for MixtureDensity layer. Passes inputs through the MixtureDensity layer and outputs a mixture of Gaussian
        distributions based on the learned parameters alpha, mu, and sigma. Mu and sigma represent the learned means and standard
        deviations, repectively, while alpha represents the mixture parameter, which determines how much of each of the num_components
        distributions are mixed in to the final output.

		:param inputs: 2D tensor, representing a batch of inputs to be called upon and passed throught the MixtureDensity layer

        :return distribution: a tensor of probability distributions
		"""

        #(Small offset values should prevent any values <=0 after relu)
        alpha = tf.math.add(self.alpha(inputs), 1e-7)
        mu = self.mu(inputs)
        sigma = tf.math.add(self.sigma(inputs), 1e-7)
        
        # Wrap mixture parameters for Mixture constructor
        cat = tfp.distributions.Categorical(probs = alpha, allow_nan_stats= False)
        
        # Generate a list of batched component distributions to pass to the Mixture constructor
        dists = [tfp.distributions.Normal(loc = mu[:, comp], scale = sigma[:, comp], allow_nan_stats=False) for comp in range(self.num_components)]

        return tfp.distributions.Mixture(cat, dists, allow_nan_stats=False)

In [ ]:
 #Mixture density model definition.
 class DensityModel(tf.keras.Model):
    
    def __init__(self, num_bins):
        """
		Intializer for a Mixture Density model. Intializes all the trainable variables and hyperparameters to be used 
        throughout the model.

		:param num_bins:  integer tensor, denoting the 1D dimension of the output. Will be a histogram with num_bins bins, and a 
                            probability distribution.
		"""

        super(DensityModel, self).__init__()
        
        # Training hyperparamaters
        self.batch_size = 256
        self.learning_rate = .001
        self.optimizer = tf.keras.optimizers.Adam(learning_rate = self.learning_rate)

        # Dense layer hyperparameters
        self.hidden_architecture = [100, 100, 100, 100]
        self.activation = tf.keras.layers.LeakyReLU(alpha=0.2)
        self.regularizer = {} #{'kernel_regularizer': keras.regularizers.l2(0.001), 'bias_regularizer': keras.regularizers.l2(0.001)}

        # Discretization parameters
        self.num_components = 12
        self.num_bins = num_bins
        self.lower_bin_val = 0
        self.upper_bin_val = 1

        # Revised for easier experimenting
        self.hidden_layers = [keras.layers.Dense(size, activation = self.activation, **self.regularizer) for size in self.hidden_architecture]
        self.mixture_density = MixtureDensity(self.num_components,self.hidden_architecture[-1])
        
        # One time bin index calculation for discretize
        self.bin_inds = tf.convert_to_tensor(np.linspace(start = self.lower_bin_val, stop = self.upper_bin_val, num = self.num_bins + 1), dtype = tf.float64)

    def discretize(self, distributions, include_tails = False):
        """
		Novel optimized discretize function for the mixture density model. Turns continuous distributions into
        discrete probability distributions by using the CDF to calculate what proportion of values falls within
        a given histogram been. Similar process to integer rounding.

		:param distributions:  tensor of tfp distributions, all the distributions to discretize by the function
		:param include_tails:  boolean, determines whether or not to include all the tail values within the discretization

        :return discrete_dist: a tensor of discretized distributions
		"""

        cdf_per_bin = tf.vectorized_map(fn = lambda bin: distributions.cdf(bin), elems = self.bin_inds)
        
        if include_tails:
            # This argument modifies the boundary bins to extend across the computed distributions' tails if true
            left_tail = tf.expand_dims(cdf_per_bin[0, :], axis = 0)
            right_tail = tf.expand_dims(1 - cdf_per_bin[-1, :], axis = 0)
            histogram_interior = tf.math.subtract(cdf_per_bin[2:-1, :], cdf_per_bin[1:-2, :])
            discrete_dist = tf.concat([left_tail, histogram_interior, right_tail], axis = 0)
        
        else:
            discrete_dist = tf.math.subtract(cdf_per_bin[1:, :], cdf_per_bin[:-1, :])
        
        return tf.transpose(discrete_dist)

    def discretize_legacy(self, distributions):
        """
		Outdated version of the discretize function described above. Discretizes a continuous probability distribution,
        but much more slowly than discretize(). It is also possibly more numerically stable and less prone to causing
        overflow errors.

		:param distributions:  tensor of tfp distributions, all the distributions to discretize by the function
        
        :return discrete_dist: a tensor of discretized distributions
		"""
        
        bins = tf.range(self.num_bins)
        binner = lambda ind:distributions.cdf(self.bin_inds[ind+1]) - distributions.cdf(self.bin_inds[ind])
        discrete_dist = tf.map_fn(fn = binner, elems = bins, fn_output_signature = tf.float32)
        
        return tf.transpose(discrete_dist)

    def call(self, input, training = False, ret_discrete = True):
        """
		Call function for MixtureDensity model. Passes input through MixtureDensity network and returns tensor of probability
        distributions. If ret_discrete is true, the discretizes model into self.num_bins bins, otherwise returns continuous 
        distributions. 

		:param input:  tensor, tensor representing multiple 1d vectors to be passed through the call function 
        :param training, boolean, indicates whether model is training or testing
        :param ret_discretem boolean, indicates whether or not to discretize the return data

        :return distribution: a tensor of discrete/continuous probability distributions
		"""
        layer_out = input
        for layer in self.hidden_layers:
            layer_out = layer(layer_out)
            #layer_out = tf.nn.dropout(layer_out, 0.2)

        distribution = self.mixture_density(layer_out)
        if ret_discrete:
            return self.discretize(distribution)
        else:
            return distribution

    def loss(self, labels, discretized_distributions, loss_type = tf.keras.losses.MSE):
        return -loss_type(labels,discretized_distributions)
        
        

In [ ]:
#Creating the Mixture Density model
MDN = DensityModel(49)

"""
Compiles all the data for training and testing the Mixture Desnity model. Creates a list of different loss functions to keep track of.
"""
losses = ["mse", "kld", "mae"]
MDN.compile(optimizer = MDN.optimizer, loss = losses[0], metrics = ["kullback_leibler_divergence","mean_squared_error", "mean_absolute_error"])

# Output name
outstr = locstr + 'Network Weights/MDN-' + str(MDN.num_components)

if not include_last:
    outstr = outstr + '-nolast'        

#Adds early stopping, if so desired
es = EarlyStopping(monitor='val_loss', verbose=1, patience=10);

# Reduce learning rate on plateau
lradj = keras.callbacks.ReduceLROnPlateau()

#Initializing necessary TensorBoard metrics for better metrics
tb_log = tf.keras.callbacks.TensorBoard(
    log_dir = locstr + "logs",
    histogram_freq = 5,
    write_graph = True,
    update_freq= 50,
    profile_batch=2,
    embeddings_freq=1)

"""
Trains and tests the mixture density network. Validation_data = data to be tested. Automatically trains and tests for 100 epochs
"""
history = MDN.fit(X_train,y_train, validation_data = (X_test,y_test), verbose = 1, batch_size = MDN.batch_size, epochs=100, callbacks = [lradj])

#Saving the weights of the mixture density model
MDN.save_weights(outstr+".h5")

Epoch 1/100
1107/1107 [==============================] - 113s 102ms/step - loss: 0.0029 - kullback_leibler_divergence: 2.6816 - mean_squared_error: 0.0029 - mean_absolute_error: 0.0187 - val_loss: 0.0020 - val_kullback_leibler_divergence: 1.9155 - val_mean_squared_error: 0.0020 - val_mean_absolute_error: 0.0145
Epoch 2/100
1107/1107 [==============================] - 101s 91ms/step - loss: 0.0013 - kullback_leibler_divergence: 0.9304 - mean_squared_error: 0.0013 - mean_absolute_error: 0.0120 - val_loss: 0.0013 - val_kullback_leibler_divergence: 0.9053 - val_mean_squared_error: 0.0013 - val_mean_absolute_error: 0.0118
Epoch 3/100
1106/1107 [============================>.] - ETA: 0s - loss: 0.0012 - kullback_leibler_divergence: 0.7289 - mean_squared_error: 0.0012 - mean_absolute_error: 0.0113

KeyboardInterrupt: ignored

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
"""
Saves all the training and testing data collected above, and writes data to a file.
"""

print(history.history)
pickle.dump(history.history, open( locstr + "Data/" + "MDN-100x100x100x100x10-MSE-FAKE.p", "wb" ))
plt.plot(log(history.history['loss']))
plt.plot(log(history.history['val_loss']))
np.min(history.history['val_loss'])

In [ ]:
#For assessing network performance, load saved model weights
model_name = ''
"""
Recompile model to load weights. Keras requires that predict() be called to initialize the model layers
"""
MDN = DensityModel(49)
MDN.compile(optimizer = MDN.optimizer, loss = losses[0], metrics = ["kullback_leibler_divergence","mean_squared_error", "mean_absolute_error"])
MDN.predict(X_test[0])

MDN = load_weights(locstr + "Network Weights/" + model_name)

In [ ]:
eval = MDN.evaluate(X_test, y_test, return_dict = True)
for name in MDN.metrics_names:
    print("%s: %f" % (name, eval[name]))

In [ ]:
# q = random.randint(0,y_test.shape[0])
# plt.semilogx(Rcent,y_test[q,:])

# test_out = MDN.predict(np.expand_dims(X_test[q,:],axis=1).T).T

# plt.semilogx(Rcent,test_out)

# print('MAE value here is ' + str(sum(abs(np.subtract(np.expand_dims(y_test[q,:],axis=1),test_out)))))

test_out = MDN.predict(X_test)

In [ ]:
#Prints out certain error metrics useful to the model
errmat = np.sum(np.abs((test_out - y_test)),axis=1)
errmat2 = np.mean((test_out - y_test)**2,axis=1)
testE = np.sum(X_test[:,:25]*dFreq.T,axis=1)
testH = X_test[:,-1]

print(errmat.shape)
print(testE.shape)
print(testH.shape)

print('SAE is: ' + str(np.nanmean(errmat)))
print(">10% SAE is: " + str(np.sum(errmat > 0.1)/len(X_test)))
print('MSE is: ' + str(np.nanmean(errmat2)))
